In [ ]:
!pip install python-dotenv
!pip install jiwer
!pip install gradio
!pip install hopsworks

In [245]:
# imports
import pandas as pd
import joblib
import numpy as np
import xgboost as xgb # XGboost
from sklearn import metrics
from sklearn.model_selection import  train_test_split,cross_val_score,KFold,GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,explained_variance_score,mean_squared_error
import warnings
import sys
import os
from time import time
warnings.filterwarnings("ignore")
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
sys.path.append('/content/drive/MyDrive/ID2223-project/air quality prediction2')
from functions import *

Mounted at /content/drive


In [ ]:
import hopsworks
project = hopsworks.login() 
fs = project.get_feature_store() 

In [223]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

In [225]:
X_train, X_test, Y_train, Y_test = feature_view.train_test_split(0.2)
X_train = X_train.drop(columns=["date"]).fillna(0)
X_test = X_test.drop(columns=["date"]).fillna(0)
model = xgb.XGBRegressor()      

In [ ]:
learning_rate = [0.002,0.004,0.006,0.008,0.01,0.03,0.05]
n_estimators = [200,400,600,800,1000,1200,1400,1600,1800,2000]
max_depth = [3,6,10,15,20]
param_grid = dict(learning_rate = learning_rate,n_estimators = n_estimators,max_depth=max_depth )#Convert to dictionary format
 
kflod=KFold(n_splits=5,shuffle=True)

print("Begin Train")
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=kflod)


grid_result = grid_search.fit(X_train, Y_train) 
print("Best: %f using %s" % (grid_result.best_score_,grid_search.best_params_))

means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))





In [ ]:
model = xgb.XGBRegressor(max_depth=6, learning_rate=0.09, n_estimators=1300,min_child_weight=5,silent=False, objective='reg:squarederror')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_pred=Y_pred.round().tolist()
Y_pred

In [ ]:
mae=mean_squared_error(Y_test, Y_pred)
mae

241.36486486486487

In [ ]:
metrics = explained_variance_score(Y_test, Y_pred, sample_weight=None, multioutput='uniform_average')
metrics

0.7275681585563579

In [ ]:
mr = project.get_model_registry()
model_dir="AIR_Forecast_Model"
if os.path.isdir(model_dir) == False:
   os.mkdir(model_dir)

joblib.dump(model, model_dir + "/AIR_Forecast_Model.pkl")   

input_schema = Schema(X_train)
output_schema = Schema(Y_train)
model_schema = ModelSchema(input_schema, output_schema)

AIR_Forecast_model = mr.sklearn.create_model(
    name="AIR_Forecast_Model", 
    metrics={"explained_variance_score" : metrics},
    model_schema=model_schema,
    description="AIR Forecast"
)

AIR_Forecast_model.save(model_dir)

In [235]:
model = mr.get_model("AIR_Forecast_Model", version=10)
model_dir = model.download()
model = joblib.load(model_dir + "/AIR_Forecast_Model.pkl")

In [236]:
Y_pred = model.predict(X_test)
Y_pred = model.predict(X_test)
Y_pred=Y_pred.round().tolist()
Y_pred
mae=mean_squared_error(Y_test, Y_pred)
mae
metrics = explained_variance_score(Y_test, Y_pred, sample_weight=None, multioutput='uniform_average')
metrics

0.9403285967756224

In [237]:
def get_weather_data_weekly(city: str, start_date: datetime) -> pd.DataFrame:
    next7days_weather=pd.read_csv('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Guangzhou/next7days?unitGroup=metric&include=days&key=5WNL2M94KKQ4R4F32LFV8DPE4&contentType=csv')
########################城市名############################
    
    df_weather = pd.DataFrame(next7days_weather)
    df_weather.rename(columns = {"datetime": "date"}, 
              inplace = True)
#########################根据模型的feature进行修改###############################
    df_weather = df_weather.drop(labels=['stations','icon','description','conditions','sunset','sunrise','severerisk','preciptype','name','feelslikemax','temp','precipprob','windspeed','cloudcover','precip','tempmax','uvindex','solarradiation','solarenergy','winddir','moonphase','snow','snowdepth'], axis=1) 

    return df_weather

In [258]:
from datetime import date
from sklearn.preprocessing import StandardScaler
today = date.today()
city = "Guangzhou"
df_weather = get_weather_data_weekly(city, today)
df_weather.date = df_weather.date.apply(timestamp_2_time)
df_weather_x = df_weather.drop(columns=["date"]).fillna(0)
df_weather_nn=np.array(df_weather_x)
scaler = StandardScaler()
scaler.fit(df_weather_x)

df_weather_use=scaler.transform(df_weather_x)

df_weather_x= pd.DataFrame(df_weather_use)

In [260]:
df_weather_x

,0,1,2,3,4,5,6,7,8
0,-0.805573,-1.095083,0.030540,0.391518,1.451542,2.645751,1.874624,-1.953656,-2.645751
1,-1.503395,-1.607245,-2.168363,-1.640143,0.613931,-0.377964,1.188642,0.707540,0.377964
2,-0.764525,-0.798568,-0.842040,-1.174554,-0.528266,-0.377964,-0.254287,1.087711,0.377964
3,-0.477187,-0.016847,-0.074169,-0.581986,-1.327804,-0.377964,-0.301596,1.045470,0.377964
4,0.343780,0.522271,0.449379,0.264539,-0.375973,-0.377964,-0.384387,0.369611,0.377964
5,0.384828,0.549226,0.588992,0.179887,-1.327804,-0.377964,-0.384387,0.285128,0.377964
6,1.287891,1.142256,0.972928,1.111064,0.271272,-0.377964,-1.614424,-0.770902,0.377964
7,1.534181,1.303992,1.042734,1.449675,1.223102,-0.377964,-0.124186,-0.770902,0.377964


In [ ]:
next7days_weather = pd.read_csv('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Guangzhou/next7days?unitGroup=metric&include=days&key=5WNL2M94KKQ4R4F32LFV8DPE4&contentType=csv')
next7days_weather

In [ ]:
df_weather = pd.DataFrame(next7days_weather)
df_weather.rename(columns = {"datetime": "date"},inplace = True)
#########################根据模型的feature进行修改###############################
df_weather = df_weather.drop(labels=['stations','icon','description','conditions','sunset','sunrise','severerisk','preciptype','name','feelslikemax','temp','precipprob','windspeed','cloudcover','precip','tempmax','uvindex','solarradiation','solarenergy','winddir','moonphase','snow','snowdepth'], axis=1) 
df_weather

In [259]:
preds = model.predict(df_weather_x)
preds

ValueError: ignored